In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Legend
output_notebook()
from bokeh.transform import factor_cmap
from bokeh.models import Legend
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.palettes import d3



Loading BokehJS ...

In [89]:


def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate",
               "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)
    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})
        all1=[]
        if name is not None:
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")
        if author is not None:
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')
        if rating is not None:
            all1.append(rating.text)
        else:
            all1.append('-1')
        if users_rated is not None:
            all1.append(users_rated.text)
        else:
            all1.append('0')     
        if price is not None:
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls


In [60]:
no_pages = 5
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')


In [61]:
df.shape
df.head(15)

,Book Name,Author,Rating,Customers_Rated,Price
0,The Psychology of Money,Morgan Housel,4.5 out of 5 stars,"6,615",₹ 290.00
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"10,903",₹ 300.00
2,World’s Greatest Books For Personal Growth & W...,Dale Carnegie,4.5 out of 5 stars,"4,097",₹ 299.00
3,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"18,055",₹ 449.00
4,Grandma's Bag of Stories: Collection of 20+ Il...,Sudha Murty,4.6 out of 5 stars,"3,931",₹ 191.00
5,My First Book of Patterns Pencil Control: Patt...,Wonder House Books,4.5 out of 5 stars,"3,864",₹ 89.00
6,Quantitative Aptitude for Competitive Examinat...,R S Aggarwal,4.4 out of 5 stars,"9,332",₹ 408.00
7,The Alchemist,Paulo Coelho,4.6 out of 5 stars,"47,429",₹ 172.00
8,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6 out of 5 stars,"6,462",₹ 595.00
9,Mathematics for Class 10 by R D Sharma (Examin...,R.D. Sharma,4.5 out of 5 stars,903,₹ 545.00


In [62]:
df["Rating"] = df["Rating"].apply(lambda x: x.split()[0])
df["Customers_Rated"] = df["Customers_Rated"].str.replace(",","")
df["Price"] = df["Price"].str.replace(",","")
df["Price"] = df["Price"].str.replace("₹","")
df["Price"] = df["Price"].apply(lambda x:x.split(".")[0])
df["Rating"] = pd.to_numeric(df["Rating"])
df["Customers_Rated"] = pd.to_numeric(df["Customers_Rated"],errors = "ignore")
df["Price"]=df["Price"].astype(int)
df.head()


,Book Name,Author,Rating,Customers_Rated,Price
0,The Psychology of Money,Morgan Housel,4.5,6615,290
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,10903,300
2,World’s Greatest Books For Personal Growth & W...,Dale Carnegie,4.5,4097,299
3,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,18055,449
4,Grandma's Bag of Stories: Collection of 20+ Il...,Sudha Murty,4.6,3931,191


In [63]:
df.dtypes

Book Name           object
Author              object
Rating             float64
Customers_Rated      int64
Price                int32
dtype: object

In [55]:
df.replace(str(0),np.nan,inplace = True)
df.replace(0,np.nan,inplace = True)

In [64]:
count_nan = len(df) - df.count()
count_nan

Book Name          0
Author             0
Rating             0
Customers_Rated    0
Price              0
dtype: int64

In [65]:
data = df.sort_values(["Price"],axis = 0 ,ascending=False)[:15]
data

,Book Name,Author,Rating,Customers_Rated,Price
75,Combo Pack: Lakhmir Singh Class 10 Science (Bi...,Manjit Kaur Lakhmir Singh,4.4,352,1180
86,New Simplified Physics : A Reference Book for ...,S.L. Arora,4.3,1193,1050
68,My First Complete Learning Library: Boxset of ...,Wonder House Books,4.6,3337,799
13,Atomic Habits: The life-changing million copy ...,James Clear,4.6,15818,600
8,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,6462,595
31,Objective NCERT at your FINGERTIPS for NEET-AI...,MTG Editorial Board,4.7,830,550
9,Mathematics for Class 10 by R D Sharma (Examin...,R.D. Sharma,4.5,903,545
92,How to Prepare for Quantitative Aptitude for t...,Arun Sharma,4.5,2111,545
32,A Modern Approach to Verbal & Non-Verbal Reaso...,R.S. Aggarwal,4.4,4029,540
54,Indian Art and Culture for Civil Services and ...,Nitin Singhania,4.6,1904,493


In [68]:
p = figure(x_range=data.iloc[:,1], plot_width=800, plot_height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")
p.vbar(x=data.iloc[:,1], top=data.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2
show(p)


In [69]:
data = df[df['Customers_Rated'] > 1000]
data = data.sort_values(['Rating'],axis=0, ascending=False)[:15]
data


,Book Name,Author,Rating,Customers_Rated,Price
99,Bhagvad gita as it is english new edition,His Divine Grace A.C. Bhaktivedanta,4.7,3570,237
15,Think Like a Monk: The secret of how to harnes...,Jay Shetty,4.7,10689,275
44,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,24454,288
62,"The Magic of the Lost Temple: Illustrated, eas...",Sudha Murty,4.7,1880,199
74,Death; An Inside Story: A book for all those w...,Sadhguru,4.7,3732,202
87,Diary of a Wimpy Kid: The Deep End (Book 15),Jeff Kinney,4.7,4690,379
21,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.7,30170,340
7,The Alchemist,Paulo Coelho,4.6,47429,172
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,10903,300
98,The Theory of Everything: The Origin and Fate ...,Stephen Hawking,4.6,4498,169


In [78]:
data = df.sort_values(["Customers_Rated"], axis=0, ascending=False)[:20]
data


,Book Name,Author,Rating,Customers_Rated,Price
84,"The Silent Patient: The record-breaking, multi...",Alex Michaelides,4.5,62629,288
7,The Alchemist,Paulo Coelho,4.6,47429,172
47,The Subtle Art of Not Giving a F*ck: A Counter...,Mark Manson,4.5,43672,292
27,Think and Grow Rich,Napoleon Hill,4.5,42583,139
60,How to Win Friends and Influence People,Dale Carnegie,4.5,40073,139
11,Rich Dad Poor Dad: What the Rich Teach Their K...,Robert T. Kiyosaki,4.6,36959,382
21,Sapiens: A Brief History of Humankind,Yuval Noah Harari,4.7,30170,340
35,Man's Search For Meaning: The classic tribute ...,Viktor E Frankl,4.5,27902,180
23,The Power of Your Subconscious Mind,Joseph Murphy,4.5,27136,109
44,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,24454,288


In [84]:
colors = itertools.cycle(palette)

palette = d3['Category20'][20]
index_cmap = factor_cmap('Author', palette=palette,
                         factors=data["Author"])
p = figure(plot_width=700, plot_height=700, title = "Top Authors: Rating vs. Customers Rated")
p.scatter('Rating','Customers_Rated',source=data,fill_alpha=0.6, fill_color=index_cmap,size=20,legend_label='Author')
p.xaxis.axis_label = 'RATING'
p.yaxis.axis_label = 'CUSTOMERS RATED'
p.legend.location = 'top_left'
show(p)